#Bibliothèque


- On importe la bibliothèque NLTK qui qui nous aidera concernant le traitement de langage naturel.
- On importe from GENSIM.MODELS les deux models Word2Vec et FastText pour crée notre propre modele.
- On importe plusieurs bibliothéques qui nous aident à réaliser notre model.   

In [ ]:
from sklearn.model_selection import train_test_split
import collections
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model
from keras.layers import *
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras import Sequential
from matplotlib import pyplot
from keras.utils.vis_utils import plot_model
import pydot
from keras import optimizers
from keras.layers import Flatten
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec, keyedvectors,FastText
from keras_preprocessing.text import text_to_word_sequence
from bs4 import BeautifulSoup
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#Les données

*Puis* en créer deux fonction de nettoyage de texte arabe

In [ ]:
def clean(text):
        # suppression des carac html
        example1 = BeautifulSoup(text, 'lxml')

        # suppression des mentions
        example2 = re.sub(r'@[A-Za-z0-9]+', '', example1.get_text())

        # suppression des liens
        ex3 = re.sub('https?://[A-Za-z0-9./]+', '', example2)

        # suppression des hashtag
        final = re.sub(r'#[A-Za-z0-9]+', "", ex3)
        def remove_emojis(data):
            emoj = re.compile("["
                          u"\U0001f600-\U0001F64F"  # emoticons
                          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                          u"\U0001F680-\U0001F6FF"  # transport & map symbols
                          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                          u"\U00002500-\U00002BEF"  # chinese char
                          u"\U00002702-\U000027B0"
                          u"\U00002702-\U000027B0"
                          u"\U000024C2-\U0001F251"
                          u"\U0001f926-\U0001f937"
                          u"\U00010000-\U0010ffff"
                          u"\u2640-\u2642"
                          u"\u2600-\u2B55"
                          u"\u200d"
                          u"\u23cf"
                          u"\u23e9"
                          u"\u231a"
                          u"\ufe0f"  # dingbats
                          u"\u3030"
                          "]+", re.UNICODE)
            return re.sub(emoj, '', data)
        texte = remove_emojis(final)
        return texte

puis on ouvre les fichiers text nécessaires et on lit chaque ligne du fichier ensuite on stocke ces lignes dans une liste, et on fait appel à les fonction de nettoyage

In [ ]:
#Answers
file = open('Shopping_Answers.txt', 'rt', encoding='utf-8')
text = file.read()
Ans = text.split("\n")
Ans[0]=Ans[0].replace('\ufeff','')
A_data = []
for A_da in Ans :
    A_data.append(clean(A_da))
#Question
file = open('Shopping_Question.txt', 'rt', encoding='utf-8')
text = file.read()
Ques = text.split("\n")
Ques[0]=Ques[0].replace('\ufeff','')
Q_data = []
for Q_da in Ques :
    Q_data.append(clean(Q_da))

#Prétraitement

Lors de cette etape ,on divisé les phrases en mots .

In [ ]:
L2 = [text_to_word_sequence(q) for q in Q_data]

Dans cette etape on creée notre modele word2vect, et à l'aide de la fonctions *save_word2vec_format* on stocke notre modèle Word2Vec sous forme de fichier texte

In [ ]:
ft_model2 = Word2Vec(sentences=L2, size=100, window=5, sample=1e-2, negative=10, sg=0, workers=10, min_count=1,iter=20)
ft_model2.wv.save_word2vec_format('SHOPPING.txt', binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


**Etape 1: TOKENIZE**


*   Son but c’est de symboliser les données, c'est à dire convertie le texte en valeurs numériques **; transformer chaque chaque mot en nombre ;** 
 

*   Cela permet au réseau neuronal d'effectuer des opérations sur les données d'entrée

In [ ]:
def tokenize(x):
    tokenizer = Tokenizer(char_level=False)
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

*   Exemple :

In [ ]:
text_sentences = ['ما هي أنواع الملابس المتوفرة للصيف','هل يوجد ملابس للصيف.']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'للصيف': 1, 'ما': 2, 'هي': 3, 'أنواع': 4, 'الملابس': 5, 'المتوفرة': 6, 'هل': 7, 'يوجد': 8, 'ملابس': 9}

Sequence 1 in x
  Input:  ما هي أنواع الملابس المتوفرة للصيف
  Output: [2, 3, 4, 5, 6, 1]
Sequence 2 in x
  Input:  هل يوجد ملابس للصيف.
  Output: [7, 8, 9, 1]


**Etape 2: PAD**


*   Lorsque nous introduisons nos séquences d'ID de mot dans le modèle finale, les séquence doient avoir la même longueur.
 

*   Pour ce faire, un remplissage est ajouté à toute séquence plus courte que la longueur maximale (c'est-à-dire plus courte que la phrase la plus longue).

In [ ]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
        print('Length Max: {}'.format(length))
        print()
    return pad_sequences(x, maxlen=length, padding='post')

*   Exemple (Poursuivre l'exemple précédent)


In [ ]:
test_pad = pad(text_tokenized)

for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Length Max: 6

Sequence 1 in x
  Input:  [2 3 4 5 6 1]
  Output: [2 3 4 5 6 1]
Sequence 2 in x
  Input:  [7 8 9 1]
  Output: [7 8 9 1 0 0]


**Etape 3: Prétraitement**


*   La création d'un dictionnaire des `{mot:id}` pour les Questions et les réponses .

*   Transfomer toutes les Questions et les Réponses à des vecteur.
*   Calcule de la taille du vocabulaire Q et A.

*   la taille maximale des Q et A

In [ ]:
def preprocess(Q,A):
    preprocess_Q, Q_tk = tokenize(Q)
    preprocess_A, A_tk = tokenize(A)
    preprocess_Q = pad(preprocess_Q)
    preprocess_A = pad(preprocess_A)
    preprocess_A = preprocess_A.reshape(*preprocess_A.shape, 1)
    return preprocess_Q, preprocess_A, Q_tk, A_tk

In [ ]:
preproc_Q, preproc_A, Q_tokenizer, A_tokenizer = preprocess(Q_data, A_data)
print(Q_tokenizer.word_index)

print(preproc_Q[:1])
max_Q_sequence_length = preproc_Q.shape[1]
max_A_sequence_length = preproc_A.shape[1]

Q_vocab_size = len(Q_tokenizer.word_index)+1
A_vocab_size = len(A_tokenizer.word_index)+1

print('Data Preprocessed')
print("Max Q sentence length:", max_Q_sequence_length)
print("Max A sentence length:", max_A_sequence_length)
print("Q vocabulary size:", Q_vocab_size)
print("A vocabulary size:", A_vocab_size)

Length Max: 12

Length Max: 23

{'هل': 1, 'يوجد': 2, 'البلوزة': 3, 'اللون': 4, 'من': 5, 'النسائية': 6, 'منها': 7, 'القميص': 8, 'الرجالية': 9, 'منه': 10, 'الشيفون': 11, 'الجاكيت': 12, 'الرجالي': 13, 'ما': 14, 'الفستان': 15, 'المقاس': 16, 'الأزرق': 17, 'الرمادية': 18, 'الزرقاء': 19, 'النسائي': 20, 'الأسود': 21, 'قياس': 22, 'لارج': 23, 'البيضاء': 24, 'مقاس': 25, 'الأحمر': 26, 'الزهري': 27, 'السوداء': 28, 'هو': 29, 'الأبيض': 30, 'الحمراء': 31, 'الأصفر': 32, 'البنطال': 33, 'الخمري': 34, 'البنفسجي': 35, 'الفاتح': 36, 'التنورة': 37, 'نوع': 38, 'اكس': 39, 'يمكنني': 40, 'الرمادي': 41, 'أصغر': 42, 'أكبر': 43, 'الخمرية': 44, 'حجز': 45, 'ميديوم': 46, 'الخضراء': 47, 'الصفراء': 48, 'الجينز': 49, 'ألوان': 50, 'الوردي': 51, 'الصوف': 52, 'سعر': 53, 'أخرى': 54, 'لون': 55, 'الغامق': 56, 'الزهرية': 57, 'الساتان': 58, 'التفاحي': 59, 'الصيفية': 60, 'البرتقالي': 61, 'سمول': 62, 'الأخضر': 63, 'الزيتي': 64, 'الشتوية': 65, 'سعرها': 66, 'الموف': 67, 'القصيرة': 68, 'المتوفرة': 69, 'القياسات': 70, 'ان': 71, 'البنف

**Etape 4: Embbedings**


*   l’utilisation d'une matrice qui contient les représentations des mots en vecteurs réalisés avec plusieurs méthodes **(W2V )** ,donc ça sera nécessaire de remplacer chaque mot par un vecteur comme input du model.


*   Et Dans notre cas la taille de chaque vecteur est 100

In [ ]:
embeddings_index = {}
f = open('SHOPPING.txt',encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()


def embedding_matrix_creater(embedding_dimention):
  embedding_matrix = np.zeros((Q_vocab_size, embedding_dimention))
  for word, i in Q_tokenizer.word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          embedding_matrix[i] = embedding_vector
  return embedding_matrix

mat=embedding_matrix_creater(100)

#Modèle



*   Division de la base de données
*   L'utlisation de GRU et CNN 



In [ ]:
input_length = max_A_sequence_length
output_length = max_Q_sequence_length
embedding_dim = 100

tmp_x = pad(preproc_Q, max_A_sequence_length)
Q_train,Q_test,A_train,A_test=train_test_split(tmp_x,preproc_A,test_size=0.1,random_state=20)

enc_in = Input(shape=(input_length,), dtype='float32', name='enc_input')

embedding_layer = Embedding(input_dim=Q_vocab_size,
                            output_dim=embedding_dim,
                            input_length=input_length,
                            trainable=False,
                            weights=[mat],
                            name='enc_embedding')

enc_embedded  =  embedding_layer(enc_in)

encoded=GRU(units=256, return_sequences=True, name='en_GRU',activation='relu')(enc_embedded)

encodedcnnf = Conv1D(128, 3, strides=1)(encoded)
encodedcnnfmaxpo = MaxPooling1D(pool_size=2,strides=2)(encodedcnnf)

encodedcnnff = Conv1D(128, 3, strides=1)(encoded)
encodedcnnfmaxpoo = MaxPooling1D(pool_size=2, strides=2)(encodedcnnff)


encodedcnnfff = Conv1D(128, 3, strides=1)(encoded)
encodedcnnfmaxpooo = MaxPooling1D(pool_size=2, strides=2)(encodedcnnfff)


encodedcnn=Concatenate(axis=1)([encodedcnnfmaxpo, encodedcnnfmaxpoo, encodedcnnfmaxpooo])

encodedcnnfff = Conv1D(256, 6, strides=1,activation='relu')(encodedcnn)
encodedcnnfmax = MaxPooling1D(pool_size=3, strides=1)(encodedcnnfff)

fc2 = TimeDistributed(Dense(units=A_vocab_size+1), name='fc2')(encodedcnnfmax)
preds = Activation('softmax', name='softmax')(fc2)

model = Model(enc_in, preds)
model.summary()



*   L'entrainement



In [ ]:
rms = optimizers.RMSprop(lr=0.001)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy',metrics=['accuracy'])

history = model.fit(Q_train,A_train, 
          epochs=60, batch_size=1024, 
          validation_split = 0.1
          )

Le réseau neuronal traduira l'entrée en ID (mot), ce qui n'est pas la forme finale que nous voulons. La fonction logits_to_text fera le pont entre les ids du réseau neuronal et les réponses.

In [ ]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

evaluation

In [ ]:
[a,b]=model.evaluate(Q_test, A_test)
print('loss : {} , accuracy: {}'.format(a,b))